In [1]:
import pickle
from stanfordcorenlp import StanfordCoreNLP
import html
import collections
import json
import re

In [2]:
angelis = pickle.load(open('ANGELIS_DATA.pkl','rb'))

In [3]:
default_properties = {'annotators': 'tokenize, pos, ner, depparse',
                      'pipelineLanguage':'en',
                      'outputFormat': 'json'}
sNLP = StanfordCoreNLP('http://localhost', port=9000)

In [4]:
parsed = sNLP.annotate('Ali lied about having to leave for her job to see if Jake would end the show to be with her .', properties = default_properties)
print(json.loads(parsed)['sentences'])
tokens = []
head = [0]*len(parsed['tokens'])
deprel = [0]*len(parsed['tokens'])
for deptoken in parsed['basicDependencies']:
    #deptoken['dependent'] <- linear index
    #deptoken['governor'] <- index of governor (head)
    #deptoken['dep'] <- dep_rel
    
    deprel[deptoken['dependent'] - 1] = deptoken['dep']
    head[deptoken['dependent'] - 1] = deptoken['governor']
print(head)
print(deprel)

[{'tokens': [{'characterOffsetEnd': 3, 'before': '', 'originalText': 'Ali', 'ner': 'PERSON', 'word': 'Ali', 'pos': 'NNP', 'index': 1, 'after': ' ', 'lemma': 'Ali', 'characterOffsetBegin': 0}, {'characterOffsetEnd': 8, 'before': ' ', 'originalText': 'lied', 'ner': 'O', 'word': 'lied', 'pos': 'VBD', 'index': 2, 'after': ' ', 'lemma': 'lie', 'characterOffsetBegin': 4}, {'characterOffsetEnd': 14, 'before': ' ', 'originalText': 'about', 'ner': 'O', 'word': 'about', 'pos': 'IN', 'index': 3, 'after': ' ', 'lemma': 'about', 'characterOffsetBegin': 9}, {'characterOffsetEnd': 21, 'before': ' ', 'originalText': 'having', 'ner': 'O', 'word': 'having', 'pos': 'VBG', 'index': 4, 'after': ' ', 'lemma': 'have', 'characterOffsetBegin': 15}, {'characterOffsetEnd': 24, 'before': ' ', 'originalText': 'to', 'ner': 'O', 'word': 'to', 'pos': 'TO', 'index': 5, 'after': ' ', 'lemma': 'to', 'characterOffsetBegin': 22}, {'characterOffsetEnd': 30, 'before': ' ', 'originalText': 'leave', 'ner': 'O', 'word': 'leave

TypeError: string indices must be integers

In [ ]:
angelis[5]

In [ ]:
def data_transform(sents):
    dataset = []
    ners = {}
    for sentence_index, sent in enumerate(sents):
        print(sentence_index + 1)
        data = {}
        id = None
        
        
        sentence = html.unescape(sent[0].strip())
        Obj = html.unescape(sent[1][1].replace('\n',' ').strip())
        Subj = html.unescape(sent[1][2].replace('\n',' ').strip())
        
        relation = 'NA' if sent[1][0] == 0 or sent[2]=='no_relation' else sent[2]
        
        tokens = []
        ner = []
        pos = []
        
        try:
            annotated = json.loads(sNLP.annotate(sentence, properties = default_properties))
        except:
            continue
        #print(annotated['sentences'][0]['tokens'])
        for sentence in annotated['sentences']:
            for analyzed_token in sentence['tokens']:
                tokens.append(analyzed_token['word'])
                ner.append(analyzed_token['ner'])
                pos.append(analyzed_token['pos'])
        for ne in ner:
            if ne not in ners:
                ners[ne] = 1
            else:
                ners[ne] += 1
        tokens_str = " ".join(tokens)
        
        
        deprel = [0] * len(tokens)
        head = [0 ] * len(tokens)
        base_index = 0
        for sentence in annotated['sentences']:
            for deptoken in sentence['basicDependencies']:
                token_index = deptoken['dependent'] - 1 + base_index
                governor_index = 0 if deptoken['governor'] == 0 else deptoken['governor'] + base_index
                
                deprel[token_index] = deptoken['dep']
                head[token_index] = governor_index
            base_index += len(sentence['tokens'])
                
        Obj_tokens = []
        Obj_ner = []
        Subj_tokens = []
        Subj_ner = []
        
        try:
            parsed_Obj = json.loads(sNLP.annotate(Obj, properties = default_properties))
            parsed_Subj = json.loads(sNLP.annotate(Subj, properties = default_properties))
        except:
            continue
        for sent in parsed_Obj['sentences']:
            for analyzed_token in sent['tokens']:
                Obj_tokens.append(analyzed_token['word'])
                Obj_ner.append(analyzed_token['ner'])
        Obj_str = " ".join(Obj_tokens)
        
        for sent in parsed_Subj['sentences']:
            for analyzed_token in sent['tokens']:
                Subj_tokens.append(analyzed_token['word'])
                Subj_ner.append(analyzed_token['ner'])
        Subj_str = " ".join(Subj_tokens)
        
        print("\'%s\' \nObj \'%s\' Subj \'%s\'"%(tokens_str, Obj_str, Subj_str))
        if Obj_str not in tokens_str:
            print("Obj \'%s\' not in \'%s\'"%(Obj_str, tokens_str))
            continue
        elif Subj_str not in tokens_str:
            print("Subj \'%s\' not in \'%s\'"%(Subj_str, tokens_str))
            continue
        
        Obj_start = -1
        Obj_end = -1
        for i in range(len(tokens)):
            check_flag = True
            for j in range(len(Obj_tokens)):
                #print(tokens[i+j])
                if tokens[i+j] != Obj_tokens[j]:
                    check_flag = False
            if check_flag:
                Obj_start = i
                Obj_end = Obj_start + len(Obj_tokens) - 1
                break
        ner_counter = collections.Counter(Obj_ner)
        Obj_type = sorted(ner_counter, key=ner_counter.get, reverse=True)[0]

        Subj_start = -1
        Subj_end = -1
        for i in range(len(tokens)):
            check_flag = True
            for j in range(len(Subj_tokens)):
                if tokens[i+j] != Subj_tokens[j]:
                    check_flag = False
            if check_flag:
                Subj_start = i
                Subj_end = Subj_start + len(Subj_tokens) - 1
                break
        ner_counter = collections.Counter(Subj_ner)
        Subj_type = sorted(ner_counter, key=ner_counter.get, reverse=True)[0]

        #print("Obj \'%s\' Subj \'%s\'"%(" ".join(tokens[Obj_start:Obj_end+1]), " ".join(tokens[Subj_start:Subj_end+1])))
        #print("Obj type \'%s\' Subj type \'%s\'"%(Obj_type, Subj_type))
        
        
        
        data['id'] = id
        data['tokens'] = tokens
        data['sentence'] = tokens_str
        data['ner'] = ner
        data['pos'] = pos
        data['head'] = head
        data['deprel'] = deprel
        data['key_start'] = Obj_start
        data['key_end'] = Obj_end
        data['key_type'] = Obj_type
        data['ans_start'] = Subj_start
        data['ans_end'] = Subj_end
        data['ans_type'] = Subj_type
        data['relation'] = 'NA' if relation == 'no_relation' else relation
        
        dataset.append(data)
    print(ners)
    return dataset

In [ ]:
dataset = data_transform(angelis)

1
'Althouse College of Education is a teacher 's college at University of Western Ontario , London , Ontario , Canada .' 
Obj 'University of Western Ontario' Subj 'London'
2
'Moulton worked for two years as an instructor at Peter Kump 's New York Cooking School -LRB- now known as the Institute of Culinary Education -RRB- , where she discovered her love of teaching .' 
Obj 'Institute of Culinary Education' Subj 'New York'
3
'Norsk Transport retained for a period maintenance of the Rjukan Line , while the Tinnos Line remained part of the rail network maintained by the Norwegian National Rail Administration .' 
Obj 'Norsk Transport' Subj 'National Rail Administration'
4
'With her two albums , '' Taylor Swift '' in 2006 and '' Fearless '' in 2008 -LRB- both on the Big Machine label -RRB- , Swift has built her career by way of Nashville and country radio .' 
Obj 'Taylor Swift' Subj 'Big Machine'
5
''' Joyce ready for great leap at Qantas '' , Sydney Morning Herald online retrieved 27 Novemb

'Cheltenham College is a co-educational independent school , located in Cheltenham , Gloucestershire , England .' 
Obj 'Cheltenham College' Subj 'England'
45
'In 1970 Hastings Banda was declared President for life of the MCP , and in 1971 Banda consolidated his power and was named President for life of Malawi itself .' 
Obj 'Hastings Banda' Subj 'Malawi'
46
47
'Maximum Pro Wrestling -LRB- MaxPro -RRB- is a Canadian independent professional wrestling organization , founded in 2010 with the merger of Scott D'Amore 's Border City Wrestling out of Windsor , Ontario and BSE Pro Wrestling out of Toronto , Ontario .' 
Obj 'Scott D'Amore' Subj 'Windsor'
48
49
'Legacy of Kain is a series of action-adventure video games primarily developed by Crystal Dynamics and published by Square Enix Europe -LRB- formerly Eidos Interactive -RRB- .' 
Obj 'Crystal Dynamics' Subj 'Europe'
50
51


In [ ]:
pickle.dump(dataset, open('Angelis.pkl','wb'))

In [ ]:
json.loads(sNLP.annotate(html.unescape(angelis[1998][0]), properties = default_properties))

In [ ]:
html.unescape(angelis[1998][0])